# Predict event type

In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn import preprocessing
import sklearn.model_selection

In [4]:
columns = ('Day', 'Date', 'Time', 'People_In', 'People_Out', 'IsEvent')

bin_data = pd.read_csv('data/building_event_binary.txt', names=columns)
print bin_data.dtypes
print bin_data.head()
bin_data.describe()

Day           object
Date          object
Time          object
People_In      int64
People_Out     int64
IsEvent       object
dtype: object
      Day      Date      Time  People_In  People_Out  IsEvent
0  Sunday  07/24/05  00:00:00          0           0  noevent
1  Sunday  07/24/05  00:30:00          1           0  noevent
2  Sunday  07/24/05  01:00:00          0           0  noevent
3  Sunday  07/24/05  01:30:00          0           0  noevent
4  Sunday  07/24/05  02:00:00          0           0  noevent


,People_In,People_Out
count,5040.000000,5040.000000
mean,3.737500,3.888294
std,6.010372,6.834565
min,0.000000,0.000000
25%,0.000000,0.000000
50%,1.000000,0.000000
75%,5.000000,5.250000
max,54.000000,62.000000
